In [ ]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt

In [ ]:
def to_seconds(str):
        splitted=str.split(':')
        return (int(splitted[0])*3600+int(splitted[1])*60+int(splitted[2]))

def string_split(str):
    a=str.split('/')
    day,month,year=int(a[0]),int(a[1]),int(a[2])
    return datetime.datetime(day=day,month=month,year=2000+year)

def month(str):
        month=str.split('/')[1]
        months={1:'January',2:'Feburary',3:'March',4:'April',5:'May',6:'June',7:'July',8:'August',9:'September',10:'October',11:'November',12:'December'}
        return months[int(month)]

def indexDates(date):
    return (date.split('/'))[0]

In [53]:
class Data:
    #This is the main dataFrame or the parent Dataframe
    df=pd.DataFrame({})
    #This is the child dataFrame derived from the first one.
    df2=pd.DataFrame({}) 
    # S={}
    MileageArray=np.array([])
    #For storing the average speeds of the given data
    Speeds=np.array([])
    #For storing all the unique dates in the given data
    dates=np.array([])
    #Storing the dataframe for each date in that month. {key=datetime object and value = dataframe}
    DayWise={}
    #It is storing the odometer readings.
    odo={}
     #In the constructor we are doing all the precalculations like:
    '''
        1. Sorting the data according to date and time
        2. Reseting the indices after the sorting
        3. Defining the Energy Column
        4. Defining the Energy Consumed Column
        5. Identifying all the unique dates and storing it in 'dates'.
    '''
    def __init__(self,dataframe):
        self.df=dataframe
        

        '''Here we are sorting the data on the basis of both date and time'''
        self.df['DateModified']=self.df['date'].map(string_split)
        self.df['Time_seconds']=self.df['time'].map(to_seconds)
        self.df.sort_values(['DateModified','Time_seconds'],inplace=True)
        
        #The distance needs to be calculated before we filter the data.
        #self.df['Distance']=self.df['odo'].diff(1).fillna(np.nan)
        self.df['Energy_Wh']=(self.df['cummulativeDischarge']*self.df['batVolt'])/1000
        self.df['EnergyConsumed']=(self.df['Energy_Wh']).diff(-1).fillna(0)
        self.df2=self.df.drop(np.append(self.df.loc[self.df['EnergyConsumed']<=0].index,[self.df.head(1).index,self.df.tail(1).index]))
        #self.df['mileage']=self.df['EnergyConsumed']/self.df['Distance']

        self.dates=self.df['date'].unique()
        self.df.reset_index(inplace=True,drop=True)
        self.df['Month']=self.df['date'].map(month) 
        # for i in self.dates:
        #     temp=np.array([],dtype=np.float64)
        #     uniqueOdometer=self.df.loc[self.df['date']==i,'odo'].unique()
        #     for j in uniqueOdometer:
        #         temp=np.append(temp,self.df[])
        self.filterData()

    #In this function we are filtering the data on the basis of the zero current values.
    def filterData(self):
        for i in self.dates:
            df3=pd.DataFrame({}) #contains the datapoints where odometer reading  is changing.
            uniqueOdo=((self.df.loc[self.df['date']==i,'odo']).unique())
            #print(uniqueOdo)
            uniqueOdoIndex=np.array([],dtype=np.int64)
            Mileage=np.array([],dtype=np.float64)
            for j in uniqueOdo:
                uniqueOdoIndex=np.append(uniqueOdoIndex,self.df.loc[(self.df['odo']==j)&(self.df['date']==i),'odo'].head(1).index)
                Mileage=np.append(Mileage,self.df2.loc[(self.df2['date']==i)&(self.df2['odo']==j)&(self.df2['EnergyConsumed']!=np.nan),'EnergyConsumed'].sum())
            df3=self.df.loc[uniqueOdoIndex]

            
            df3['Distance']=(df3['odo'].diff(1)).fillna(-1)
            df3['TimeDifference_seconds']=(df3['Time_seconds'].diff(1).fillna(-1))
    
            #Here we are calculating the mileage by finding the net energy spent in that day.
            Mileage
            df3['mileage']=pd.Series(Mileage)
            df3['AvgSpeed']=df3['Distance']/(df3['TimeDifference_seconds']/3600)
            df3.reset_index(drop=True,inplace=True)
            self.DayWise[i]=df3
    

In [ ]:
columns_to_pick=['date','time','speed','odo','batVolt','batCurrent','cummulativeCharge','cummulativeDischarge','chargeStatus','dischargeStatus']
df=pd.read_csv('1monthdata.csv',usecols=columns_to_pick)
df

In [54]:
OneMonth=Data(df)
#OneMonth.df2

In [ ]:
OneMonth.DayWise['01/02/23']

In [ ]:
uniqueOdo=type((OneMonth.df['odo'][df['date']=='01/02/23']).unique()[0])
uniqueOdo
# uniqueOdoIndex=np.array([])
# for j in uniqueOdo:
#     uniqueOdoIndex=np.append(uniqueOdoIndex,(OneMonth.df['odo'][OneMonth.df['odo']==j]).head(1))
#     print(uniqueOdoIndex)

In [ ]:
((OneMonth.df['odo'][OneMonth.df['date']=='01/02/23']).unique())

In [ ]:
OneMonth.DayWise['01/02/23'].loc[2,'mileage']

In [ ]:
OneMonth.df2

In [ ]:
plt.plot(OneMonth.DayWise['01/02/23'].loc[1:,'AvgSpeed'],color='red')
AvgSpeed=OneMonth.DayWise['01/02/23'].loc[1:,'AvgSpeed'].mean()
# plt.plot(20,AvgSpeed,color='black')